In [19]:
import  glob

def log_file_path(memtable_size,cpu_count,storage_device="StorageMaterial.NVMeSSD",prefix="Resources/"):
    target_path = prefix
    
    target_path += "%s/%s/%s" % (storage_device,
               str(cpu_count)+"CPU",
               str(memtable_size)+"MB")
    return target_path
# you can test it like this

def find_log_file(log_dir):
    log_file = [filename for filename in glob.glob(
                log_dir+"/**") if "LOG" in filename][0]    
    return log_file

dir_pres = ["nvme_increasingCPUs_flexibleSST"]
prefixes = ["Resources/NVMeProblems/" + dir_pre + "/45GB/" for dir_pre in dir_pres]
materials = ['NVMeSSD']
cpu_count_array = [4,8,12]
memtable_size_array = [64]

log_files = []

for prefix in prefixes:
    for material in materials: 
        for cpu_count in cpu_count_array:
            for memtable_size in memtable_size_array:
                log_dir = log_file_path(memtable_size,cpu_count,
                                      "StorageMaterial."+material,prefix)
                log_files.append(find_log_file(log_dir))

print(log_files)

Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/4CPU/64MB
Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/8CPU/64MB
Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/12CPU/64MB
['Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/4CPU/64MB/LOG_1290', 'Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/8CPU/64MB/LOG_1330', 'Resources/NVMeProblems/nvme_increasingCPUs_flexibleSST/45GB/StorageMaterial.NVMeSSD/12CPU/64MB/LOG_1310']


Analyze the data from increasing CPU

In [20]:
import re
import json
job_id_dict = {}
event_row_dict = {}
EVENTS_NEED_COLLECTED = ['compaction_started','compaction_finished']


def handle_single_file(file):
    job_dict = {}
    event_dict = {}
    
    for event in EVENTS_NEED_COLLECTED:
        event_dict[event] = {}
        job_dict[event] = []
        
    file_content = open(file,"r").readlines()
    
    for line in file_content:
        line = re.search('(\{.+\})', line)
        if line:
            try:
                log_row = json.loads(line[0])
                job_id = int(log_row['job'])
                current_event = log_row['event']
                if current_event in EVENTS_NEED_COLLECTED:
                    if job_id not in job_dict[current_event]:
                        event_dict[current_event][job_id] = [log_row]
                        job_dict[current_event].append(job_id)
                    else:
                        event_dict[current_event][job_id].append(log_row)
                        print(log_row)
            except json.decoder.JSONDecodeError:
                pass
    return job_dict,event_dict

job_id_dict,event_row_dict = handle_single_file(log_files[0]) 

print(job_id_dict.keys())
print(event_row_dict.keys())

dict_keys(['compaction_started', 'compaction_finished'])
dict_keys(['compaction_started', 'compaction_finished'])


In [31]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from statistics import mean

def cpu_time_plot_single_file(result_prefix,job_dict,event_dict):
    compaction_reason_dict = {}
    compaction_latencies = []
    compaction_cpu_time = []
    # group the compaction according to the reason
    for compaction_job in job_dict['compaction_started']:    
        compaction_start_row = event_dict['compaction_started'][compaction_job][0]
        compaction_finish_row = event_dict['compaction_finished'][compaction_job][0]
        current_reason = compaction_start_row['compaction_reason']
        
        if current_reason not in compaction_reason_dict:
            compaction_reason_dict[current_reason] = []
        else:
            compaction_reason_dict[current_reason].append(compaction_job)
        compaction_latencies.append(compaction_finish_row['compaction_time_micros'])
        compaction_cpu_time.append(compaction_finish_row['compaction_time_cpu_micros'])
    print(result_prefix)
    print("Average Compaction Latency", mean(compaction_latencies))
    print("Average CPU Time", mean(compaction_latencies))
    print("Average ")
        
    # fig = plt.figure(figsize=(16, 9))
    # fig.suptitle("Overall Time")

for log_file in log_files:
    current_job_dict, current_event_dict = handle_single_file(log_file)
    figure_prefix = log_file.replace("/","+")
    cpu_time_plot_single_file(str(figure_prefix),current_job_dict,current_event_dict)
    

Resources+NVMeProblems+nvme_increasingCPUs_flexibleSST+45GB+StorageMaterial.NVMeSSD+4CPU+64MB+LOG_1290
Average Compaction Latency 5965902.233576642
Average CPU Time 5965902.233576642
Resources+NVMeProblems+nvme_increasingCPUs_flexibleSST+45GB+StorageMaterial.NVMeSSD+8CPU+64MB+LOG_1330
Average Compaction Latency 6010740.337860781
Average CPU Time 6010740.337860781
Resources+NVMeProblems+nvme_increasingCPUs_flexibleSST+45GB+StorageMaterial.NVMeSSD+12CPU+64MB+LOG_1310
Average Compaction Latency 6426537.803056027
Average CPU Time 6426537.803056027


According to the above result, adding CPUs will slightly increase the compaction time, but it won't hurt too much on NVMe SSD.

This might be explained by NVMe behaves better with deep queue depth